In [9]:
# main library required is transformer with pytorch backend
# no tensorflow

!pip install transformers

     |████████████████████████████████| 4.0 MB 11.9 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 52.7 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
     |████████████████████████████████| 6.6 MB 42.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
# loading the final reranked file which should query and document
import pandas as pd
stance_df = pd.read_csv('/content/drive/MyDrive/Touche/processed-touche22-task2-stance-dataset.tsv.txt', sep='\t')

In [4]:
#subsetting the final reranked file
stance_df= stance_df.iloc[:100,:]
stance_df.head()

,ds,id,question,answer,answer_stance,answer_stance_object,object_count,object_1,mask_pos_1,object_2,mask_pos_2
0,softwareengineering.stackexchange,10373,What's the better way to charge for a cloud pl...,"Like all good questions, the answer depends. I...",2,simple but more expensive,2,simple but more expensive,"[[231, 252], [389, 393], [605, 619], [753, 757]]",complicated but cheaper,[]
1,softwareengineering.stackexchange,18838,Haskell AND Lisp vs. Haskell OR Lisp,"I suggest learning both, Haskell first, then C...",2,Haskell AND Lisp,2,Haskell AND Lisp,"[[25, 56]]",Haskell OR Lisp,[]
2,softwareengineering.stackexchange,19392,When is it better to offload work to the RDBMS...,You want to do all set-based operations in the...,3,do it in code,2,offload work to the RDBMS,"[[40, 55], [232, 248]]",do it in code,"[[165, 175], [578, 598]]"
3,softwareengineering.stackexchange,20653,Is it better to specialize in a single field I...,"Specialise if you enjoy it As you are aware, ...",3,expand into other fields to broaden my horizons,2,to specialize in a single field I like,"[[53, 63], [404, 410], [512, 519]]",expand into other fields to broaden my horizons,"[[892, 933]]"
4,softwareengineering.stackexchange,21186,Microsoft SDE Interview vs Microsoft SDET Inte...,"Unfortunately, those are both myths. SDEs and ...",1,Neutral,2,Microsoft SDE Interview,"[[37, 41], [283, 295], [950, 962]]",Microsoft SDET Interview,"[[46, 51], [341, 354], [832, 836], [938, 945]]"


In [30]:
# input for stance prediction models (Functionaly stance model takes question and answer as input, like strings)
from sklearn.utils import shuffle
dataset_df = shuffle(stance_df[['question','answer',]])
dataset_df.head()

,question,answer
7,Office arangement - comfort vs. teamwork?,Get some chat software installed Aside fr...
32,"Building websites, which is the better approac...",I guess you're asking in the context of the Mi...
79,What are the trade-offs of working in the offi...,I think it's important to set boundaries with ...
54,Which is better for back pain-- ice or heat?,Looks like for the first 72 hours after an bac...
5,How is IntelliJ better than Eclipse?,I work with Intellij (9.0.4 Ultimate) and Ecli...


In [10]:
# import statements for stance tasks.
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

In [11]:
# make sure, that we have cpu only for TIRA submissions.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device "{device}"')

Using device "cpu"


In [24]:
# model output 1: for object/neutral_no separator model.
# it separates first and second model from the neutral/no stance
def object_separator(evidence, claim, tokenizer, model):
    model.eval()
    def encode(claim, rationale):
        encoding = tokenizer(claim, rationale, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        return input_ids, attention_mask

    def predict(model, evidence, claim):
        with torch.no_grad():
            input_ids, attention_mask = encode(claim, evidence)
            logits = model(input_ids.to(device)).logits
            output = logits.argmax(dim=1).tolist()[0]
        return output

    return predict(model, evidence, claim)

In [25]:
# model output 1: for object/neutral_no detector model.
# it predicts first and second models as outputs.
def object_detector(evidence, claim, tokenizer, model):
    model.eval()
    def encode(claim, rationale):
        encoding = tokenizer(claim, rationale, padding=True, truncation=True, max_length=512, return_tensors="pt")
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        return input_ids, attention_mask

    def predict(model, evidence, claim):
        with torch.no_grad():
            input_ids, attention_mask = encode(claim, evidence)
            logits = model(input_ids.to(device)).logits
            output = logits.argmax(dim=1).tolist()[0]
        return output

    return predict(model, evidence, claim)

In [27]:
# this function has the logic for converting prediction numbers to labels.
# please, take time and try to understand it.
# also, feel free to call if required for understanding.
def detect(evidence, claim, tokenizer_n, model_n, tokenizer_e, model_e):
    # first step
    # predicting neutral or not
    first = object_separator(evidence, claim, tokenizer_n, model_n)
    if first == 1:
        final = 1
    elif first == 0:
        final = 0
    else:
        #predicting support or not
        second = object_detector(evidence, claim, tokenizer_e, model_e)
        if second == 0:
            final = 2
        elif second == 1:
            final = 3
    return final

In [21]:
# loading andreea's models into the 'n' and 's' two-step models.
model_n_path = "/content/drive/MyDrive/Touche/neutral_no_object_classifier-epoch-10-f1-5725"
tokenizer_n = AutoTokenizer.from_pretrained(model_n_path)
config_n = AutoConfig.from_pretrained(model_n_path, num_labels=3)
model_n = AutoModelForSequenceClassification.from_pretrained(model_n_path, config=config_n).to(device)

model_s_path = "/content/drive/MyDrive/Touche/OBJECT1_OBJECT2-CLASSIFIER"

tokenizer_s = AutoTokenizer.from_pretrained(model_s_path)
config_s = AutoConfig.from_pretrained(model_s_path, num_labels=3)
model_s = AutoModelForSequenceClassification.from_pretrained(model_s_path, config=config_s).to(device)
# num_classes are 3 for a reason, please contact, don't modify

In [16]:
# for conversion in below code
LABELS = ['NO', 'NEUTRAL', 'FIRST', 'SECOND']

In [17]:
# this is just added, for writing the output as jsonlines
# you can remove it if required
!pip install jsonlines

In [19]:
import jsonlines
output = jsonlines.open('stance_prediction_output.jsonl', 'w')

In [29]:
# stance prediction and output writing code.
for q_, d_ in tqdm(list(zip(dataset_df['question'], dataset_df['answer']))):
    label_index = detect(d_, q_, tokenizer_n, model_n, tokenizer_s, model_s)
    label = LABELS[label_index]
    print('query :'+ q_)
    print('document :'+ d_)
    print(label)
    print('\n')
    output.write({
        'query': q_,
        'answers': d_,
        'labels': label
    })

  1%|          | 1/100 [00:07<11:49,  7.17s/it]

query :Differences between cooking a whole duck vs chicken or turkey?
document :With chicken and turkey, the most important "trick" to cooking it is to make sure the dark meat gets done before the white meat dries out, and to make sure the skin crisps up somewhat.  Duck is all dark meat, and has a thick layer of fat that must be rendered out. There is not a lot of danger in drying out the breast meat like with a chicken.  Like Martha said, it's best to make a few shallow cuts in the skin over the breast (don't go all the way through to the meat) to help the fat render out. A simple (western) roasted bird would be cooked at 350°F for about 1 hour 45 min, with the oven turned up to 500°F for another 15 minutes to crisp up the skin. There is a LOT of fat rendered out, so it's best to roast in a sturdy roasting pan, on a rack (so it doesn't sit in the fat), and drain the fat about an hour into cooking (save the fat, though; it's delicious).  Personally, I think duck is easier to cook, but 

  2%|▏         | 2/100 [00:12<09:31,  5.83s/it]

query :Is it better for a mathematical paper to have a citation or a nice theorem + another coauthor?
document :I think it is not important which option makes your paper look good, but the best question here is "what is the right thing to do?"  If you are such a talented and hardworking undergraduate student who can write a publishable paper, you will certainly have a very bright future in grad school and later as a researcher. So, don't worry how your first paper is going to be evaluated or cited. Hopefully you will write better papers and you can prove yourself in the future. Regarding publishing research papers, the right thing to do is that you publish your paper as perfect as it is possible for you at the moment, which means it is better you add the solution (and one more co-author) to your paper.  
SECOND




  3%|▎         | 3/100 [00:14<07:02,  4.36s/it]

...

query :How is IntelliJ better than Eclipse?
document :I work with Intellij (9.0.4 Ultimate) and Eclipse (Helios) every day and Intellij beats Eclipse every time.   How? Because Intellij indexes the world and everything just works intuitively. I can navigate around my code base much, much faster in Intellij. F3 (type definition) works on everything - Java, JavaScript, XML, XSD, Android, Spring contexts. Refactoring works everywhere and is totally reliable (I've had issues with Eclipse messing up my source in strange ways). CTRL+G (where used) works everywhere. CTRL+T (implementations) keeps track of the most common instances that I use and shows them first.   Code completion and renaming suggestions are so clever that it's only when you go back to Eclipse that you realise how much it was doing for you. For example, consider reading a resource from the classpath by typing getResourceAsStream("/ at this point Intellij will be showing you a list of possible files that are currently availab

  9%|▉         | 9/100 [00:42<07:48,  5.15s/it]

query :which is better windows 2000 server or advanced server?
document :I feel that Advanced Server is better because it gives you many more customizations. However, if your computer is older.... Windows 2000 Server should do fine.
NEUTRAL




  9%|▉         | 9/100 [00:48<08:06,  5.34s/it]


KeyboardInterrupt: ignored